In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.options import Options

usr = "dr.gasparjan@gmail.com"
pwd = "00150740"
query = 'swiss posters magazine'
no_of_pins = 100
step = 10
headless_mode = True

options = Options()
if headless_mode:
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')

driver = webdriver.Chrome(chrome_options=options)
driver.get('https://pinterest.com/')
time.sleep(3)

elem = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div[3]/div/div[2]/div[2]/button')
elem.click()
time.sleep(3)

elem = driver.find_element_by_id("email")
elem.send_keys(usr)
time.sleep(1)
elem = driver.find_element_by_id("password")
elem.send_keys(pwd)
time.sleep(1)
elem.send_keys(Keys.RETURN)

time.sleep(5)
elem = driver.find_element_by_xpath('/html/body/div[1]/div/div[1]/div/div/div/div[2]/div[1]/div/div[2]/div/div/div[2]/div/div/div/div/div[1]/div[2]/input')
elem.send_keys(query)
time.sleep(3)
elem.send_keys(Keys.RETURN)
time.sleep(5)

elem = driver.find_element_by_tag_name("body")
pins_list = set()

while no_of_pins > 0:
    post_elems = driver.find_elements_by_class_name('pinWrapper')
    prev_pins_len = len(pins_list)
    for pin in post_elems:
        try:
            pin_link = pin.find_element_by_tag_name('a').get_attribute('href')
        except:
            pass
        pins_list.update([pin_link])
    diff = len(pins_list) - prev_pins_len
    if diff:
        no_of_pins -= diff
        for i in range(step):
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
    else: 
        break
pins_list = list(pins_list)

driver.close()

In [2]:
len(pins_list)

100

In [27]:
import requests
import os
from urllib.request import urlretrieve

from tqdm import tqdm
from bs4 import BeautifulSoup

class PinterestAPI:
    def __init__(self, path=None, access_token=None):
        self.path = path
        self.access_token = access_token 

        if not os.path.isdir(self.path):
            os.makedirs(self.path)
            
    def save_image(self, url, i):
        extension = url.split('.')[-1]
        saved_image_path = urlretrieve(url, "{}/{}.{}".format(self.path, str(i), extension))

    def get_image_url_from_pin_url(self, url):
        r = requests.get(url, timeout=10)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            images = [x['src'] for x in soup.findAll('img')]
            if len(images) > 0:
                return images[0]
            return False
        else:
            raise ValueError("Can't get image from pin {} because of returning {} error code.".format(url, r.status_code))
    
    def get_pins_from_board(self, boardId):
        response = requests.get('https://api.pinterest.com/v1/boards/'+boardId+'/pins/', timeout=10,
                                params={'access_token':self.access_token, 'page_size':10e6})
        imageDatas = response.json()['data']
        for i, imageData in enumerate(tqdm(imageDatas)):
            image_url = self.get_image_url_from_pin_url(imageData["url"])
            if image_url:
                self.save_image(image_url, i)
                
    def get_pins(self, pins_list):
        if type(pins_list) != list:
            pins_list = list(pins_list)
        for i, pin in enumerate(tqdm(pins_list)):
            response = requests.get('https://api.pinterest.com/v1/pins/'+pin.split('/')[-2], timeout=10,
                                    params={'access_token':self.access_token})
            imageData = response.json()['data']['url']
            image_url = self.get_image_url_from_pin_url(imageData)
            if image_url:
                self.save_image(image_url, i)

In [ ]:
accessToken = "Ae68KEq8bIai-7vSi_oqm7_OLn7_FRntp6Mu-QFEwj0mj8A5WQAAAAA"
boardId = "18929329622858791"
folderPath = "./pinterest_images"

getter = PinterestAPI(path=folderPath, access_token=accessToken)
getter.get_pins(pins_list)